## Import Statements

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import seaborn as sns
from scipy import stats

pd.options.display.max_columns=25

## Data FY 2014

In [2]:
data_2014 = pd.read_excel('houston-houston-electricity-bills/coh-fy2014-ee-bills-july2013-june2014.xlsx')
orig_shape_2014 = data_2014.shape[0]

data_2014.shape

(67838, 24)

In [3]:
data_2014.head(5)

Reliant Contract No Service Address  Meter No                    ESID  \
0              2059605   10518 BELLAIRE   303261  1008901000140050014100   
1              2059605   10518 BELLAIRE   303261  1008901000140050014100   
2              2059605   10518 BELLAIRE   303261  1008901000140050014100   
3              2059605   10518 BELLAIRE   303261  1008901000140050014100   
4              2059605   10518 BELLAIRE   303261  1008901000140050014100   

   Business Area  Cost Center  Fund Bill Type  Bill Date  Read Date  \
0           2000   2000040005  8300         T 2014-05-28 2014-05-21   
1           2000   2000040005  8300         T 2014-04-28 2014-04-22   
2           2000   2000040005  8300         T 2014-03-27 2014-03-23   
3           2000   2000040005  8300         T 2014-02-27 2014-02-23   
4           2000   2000040005  8300         T 2014-01-27 2014-01-21   

    Due Date  Meter Read  Base Cost ($)  T&D Discretionary ($)  \
0 2014-06-27     64981.0       36098.63                2600.53   
1 2014-05-28     64299.0       38619.91                2576.95   
2 2014-04-26     63569.0       36275.51                2538.54   
3 2014-03-29     62884.0       43283.31                2207.88   
4 2014-02-26     62066.0       37987.10                1992.16   

   T&D Charges ($)  Current Due ($)  Index Charge ($)  Total Due ($)  \
0         10078.92         50249.89               0.0       50249.89   
1          9978.70         52378.23               0.0       52378.23   
2          9896.25         49788.85               0.0       49788.85   
3          9373.41         56056.70               0.0       56056.70   
4          8762.55         49748.12               0.0       49748.12   

   Franchise Fee ($) Voucher Date  Billed Demand (KVA)  kWh Usage  \
0           -1014.83   2014-05-29          1470.526316     818192   
1           -1005.86   2014-04-29          1442.268041     875338   
2            -981.00   2014-03-28          1425.000000     822201   
3           -1053.49   2014-02-28          1513.265306     981036   
4            -950.63   2014-01-30          1373.195876     860995   

   Nodal Cu Charge ($)  Adder Charge ($)  
0               479.19               0.0  
1               161.23               0.0  
2                93.04               0.0  
3                77.36               0.0  
4                17.73               0.0

### Checking Nulls

In [4]:
data_2014.isna().sum()

Reliant Contract No          0
Service Address              0
Meter No                 17633
ESID                         0
Business Area                0
Cost Center                  0
Fund                         0
Bill Type                    0
Bill Date                    0
Read Date                    0
Due Date                     0
Meter Read                  16
Base Cost ($)                6
T&D Discretionary ($)        0
T&D Charges ($)              0
Current Due ($)              0
Index Charge ($)             3
Total Due ($)                0
Franchise Fee ($)            0
Voucher Date                 0
Billed Demand (KVA)         16
kWh Usage                    0
Nodal Cu Charge ($)          7
Adder Charge ($)             3
dtype: int64

### Checking Index Charge ($) column - This was previously Adjustment

In [5]:
data_2014['Index Charge ($)'].value_counts(dropna=False)

0.00        67320
NaN             3
8.23            3
8.04            2
7.36            2
            ...  
13587.81        1
10639.12        1
9521.45         1
93.97           1
70.22           1
Name: Index Charge ($), Length: 511, dtype: int64

The column does does have information regarding a certain price. Since we are using the total due amount at the end, Index Charge ($) does not need to be present again, as it would be included in the total due amount.

In [6]:
data_2014.drop(columns=['Index Charge ($)'], inplace=True)

### Checking Unique Number of Customers

There are quite a few columns in the dataset that signify relating to a unique person/house/business. Checking the unique counts of such columns.

In [7]:
check_unique_columns = ['Reliant Contract No', 'Service Address ', 'Meter No', 
                        'ESID', 'Business Area', 'Cost Center',]

for col in check_unique_columns:
    print(f'Number of Unique Values in {col}: {data_2014[col].nunique()}')

Number of Unique Values in Reliant Contract No: 5888
Number of Unique Values in Service Address : 5824
Number of Unique Values in Meter No: 4026
Number of Unique Values in ESID: 5885
Number of Unique Values in Business Area: 8
Number of Unique Values in Cost Center: 34


NOTE: Compared to previous years, there is one less business area.

Based on the above reported values and further research online:

ESID signifies a unique ID provided to each customer subscribed to the electricity board. It would be best to choose ESID and Service Address columns going forward as these would provide number of unique customers and the areas (streets) where higher usage of electricity occurs.

Business Area signifies a grouping a number of buildings which covers a certain area. This would be useful usage patterns grouped by certain zones in the city.

### Checking Bill Type

In [8]:
data_2014['Bill Type'].value_counts(dropna=False)

T    67340
P      498
Name: Bill Type, dtype: int64

Bill Type could signify the type of the connection given. Since commercial, residential and government spaces would have different type of pricing and needs this column could be capturing that information.

Previously there were 3 types of Bills. T, P, and C. But in year 2014 there are only 2 types. 

In [9]:
data_2014['Service Address '].nunique(), data_2014['Meter No'].nunique(), data_2014['ESID'].nunique()

(5824, 4026, 5885)

The next 3 columns are: Bill Date, Read Date and Due Date. Of these it would be best to choose the Bill date across all the data files to keep the data consistent. 

### Electricity Usage Statistics

In [10]:
data_2014[['Meter Read', 'Billed Demand (KVA)', 'kWh Usage']].describe()

Meter Read  Billed Demand (KVA)     kWh Usage
count   67822.000000         67822.000000  6.783800e+04
mean    11922.180792            41.816244  1.757642e+04
std     19950.210597           365.658193  1.925858e+05
min         0.000000             0.000000  0.000000e+00
25%         0.000000             0.000000  1.000000e+00
50%      4507.000000             0.000000  2.200000e+02
75%     12230.750000             7.000000  1.419750e+03
max    492196.000000         17348.148148  9.383361e+06

There are 3 columns that denote the amount of electricity: Meter Read, Billed Demand, kWh Usage.

Using kWh Usage as a standard unit of measurement.

In [11]:
data_2014[[
    'Base Cost ($)', 'T&D Discretionary ($)', 'T&D Charges ($)', 
    'Current Due ($)', 'Total Due ($)', 'Franchise Fee ($)', 
    'Nodal Cu Charge ($)', 'Adder Charge ($)'
     ]].describe()

Base Cost ($)  T&D Discretionary ($)  T&D Charges ($)  Current Due ($)  \
count   67832.000000           67838.000000     67838.000000     67838.000000   
mean      752.493762             346.176442       243.271038      1429.004427   
std      8850.260472           11412.058754      1761.077978     17419.486301   
min     -1269.920000             -45.930000     -1715.790000      -104.790000   
25%         0.070000               3.140000         5.750000        12.080000   
50%        10.020000               6.440000        10.380000        27.330000   
75%        59.690000              17.800000        60.692500       164.850000   
max    586509.580000          781792.760000     70591.250000    906606.680000   

       Total Due ($)  Franchise Fee ($)  Nodal Cu Charge ($)  Adder Charge ($)  
count   67838.000000       67838.000000         67831.000000      67835.000000  
mean     1428.948580         -30.802815             2.847491          8.410448  
std     17420.904924         210.583496            42.663730        276.832133  
min         0.000000       -6179.070000            -0.010000          0.000000  
25%        12.080000          -4.830000             0.000000          0.000000  
50%        27.270000          -0.470000             0.030000          0.000000  
75%       164.212500           0.000000             0.170000          0.000000  
max    906606.680000           0.000000          5531.690000      12528.970000

Adder Charge ($) does not contain any useful information. Electing to drop that column. Previously this column was Reliability Unit Charge.

The columns other than Current Due or Total Due are adding up the value present in these two columns. Going forward choosing the column Total Due ($). 
Based on the above statistics the columns Current Due and Total Due represent the same value. 

Based on the above analysis of the dataset choosing the following columns:

1. ESID
2. Business Area
3. Service Address 
3. Bill Type
4. Bill Date
5. Total Due ($)
6. kWh Usage

### Selecting and Filtering Columns

In [12]:
data_2014 = data_2014[[
    'ESID', 'Business Area', 'Service Address ', 'Bill Type',
    'Bill Date', 'Total Due ($)', 'kWh Usage'
]]

In [13]:
rename_cols = {
    'ESID': 'esid',
    'Business Area': 'business_area',
    'Service Address ': 'service_address',
    'Bill Type': 'bill_type',
    'Bill Date': 'bill_date',
    'Total Due ($)': 'total_due',
    'kWh Usage': 'kwh_usage'
}

data_2014_main = data_2014.rename(columns=rename_cols)

Checking for Nulls again and dtypes

In [14]:
data_2014_main.isna().sum()

esid               0
business_area      0
service_address    0
bill_type          0
bill_date          0
total_due          0
kwh_usage          0
dtype: int64

In [15]:
data_2014_main.dtypes

esid                       object
business_area               int64
service_address            object
bill_type                  object
bill_date          datetime64[ns]
total_due                 float64
kwh_usage                   int64
dtype: object

In [16]:
data_2014_main.shape

(67838, 7)

In [17]:
zscore_2014 = stats.zscore(data_2014_main[['total_due', 'kwh_usage']])

zscore_2014

total_due  kwh_usage
0       2.802455   4.157220
1       2.924627   4.453952
2       2.775990   4.178037
3       3.135782   5.002792
4       2.773652   4.379476
...          ...        ...
67833  -0.044563  -0.078056
67834   0.024122  -0.078056
67835  -0.047622  -0.078056
67836  -0.047384  -0.078056
67837  -0.043133  -0.078056

[67838 rows x 2 columns]

Each zscore value signifies how many standard deviations away an individual value is from the mean. This is a good indicator to finding outliers in the dataframe.

Usually z-score=3 is considered as a cut-off value to set the limit. Therefore, any z-score greater than +3 or less than -3 is considered as outlier which is pretty much similar to standard deviation method

In [18]:
# data_2014_main = data_2014_main[(np.abs(zscore_2014) < 3).all(axis=1)]

data_2014_main.shape

(67838, 7)

The number of rows has decreased from 67,838 to 67,427. So 411 rows were outliers based on the data.

In [19]:
data_2014_main.head(5)

esid  business_area service_address bill_type  bill_date  \
0  1008901000140050014100           2000  10518 BELLAIRE         T 2014-05-28   
1  1008901000140050014100           2000  10518 BELLAIRE         T 2014-04-28   
2  1008901000140050014100           2000  10518 BELLAIRE         T 2014-03-27   
3  1008901000140050014100           2000  10518 BELLAIRE         T 2014-02-27   
4  1008901000140050014100           2000  10518 BELLAIRE         T 2014-01-27   

   total_due  kwh_usage  
0   50249.89     818192  
1   52378.23     875338  
2   49788.85     822201  
3   56056.70     981036  
4   49748.12     860995

In [20]:
orig_shape_2014 - data_2014_main.shape[0]

0

In [21]:
data_2014_main.to_csv('electricity_usage_data_2014.csv', index=False)